In [3]:
import pandas as pd

def read_data(fname, num_sent=4):
    contexts = []
    endings = []
    labels = []
    data = pd.read_csv(fname)
    for idx, row in data.iterrows():
        sents = []
        for i in [4,3,2,1]:
            if len(sents) == num_sent:
                break
            sents.insert(0, row[f'sentence_{i}'].strip())
        context = ' '.join(sents) # row['Kalimat-1'] +' '+ row['Kalimat-2'] +' '+ row['Kalimat-3'] +' '+ row['Kalimat-4']
        ending1 = row['correct_ending'].strip()
        ending2 = row['incorrect_ending'].strip()
        
        contexts.append(context)
        endings.append(ending1)
        labels.append(1)
        
        contexts.append(context)
        endings.append(ending2)
        labels.append(0)
    return contexts, endings, labels

In [4]:
import fasttext

fasttext_model = fasttext.load_model("cc.su.300.bin")

In [11]:
from numpy import dot
from numpy.linalg import norm
import numpy as np


def cos_sim(a,b):
    return dot(a, b)/(norm(a)*norm(b))

def get_score(sent1, sent2):
    arr1 = np.zeros(300)
    for word in sent1.split():
        arr1+=fasttext_model.get_word_vector(word.lower())
    
    arr2 = np.zeros(300)
    for word in sent2.split():
        arr2+=fasttext_model.get_word_vector(word.lower())
    arr1 = arr1 / len(sent1.split())       
    arr2 = arr2 / len(sent2.split())
    return cos_sim(arr1, arr2)

In [14]:
for num_sent in [1,2,3,4]:
    scores = []; true = 0; idy = 0
    contexts, endings, _ = read_data('../../dataset/test_su_dummy.csv', num_sent)
    for idx, context in enumerate(contexts):
        scores.append(get_score(context, endings[idx]))
    while idy < len(scores):
        if scores[idy] > scores[idy+1]:
            true+=1        
        idy+=2
    print(num_sent, true/(len(contexts)/2))

1 0.592
2 0.596
3 0.64
4 0.648
